# Logs-of-user-actions

Есть таблица logs действий пользователей (user_id, event, event_time,value). Действия пользователей естественно поделить на сессии - последовательности событий, в которых между соседними по времени событиями промежуток не более 5 минут. То есть длина всей сессии может быть гораздо больше 5 минут, но между каждыми последовательными событиями не должно быть более 5 минут.

In [1]:
from datetime import datetime  # для заполнения таблицы
import os.path  # для проверки существования файла базы данных
import sqlite3  # для работы с таблицей
import pandas as pd  # для просмотра таблицы
from datetime import timedelta
from random import randrange
import numpy as np

Поле event может принимать разные значения, в том числе ’template_selected’ (пользователь выбрал некий шаблон). В случае, если event = ’template_selected’, то в value записано название этого шаблона (например, ’pop_art_style’). Я написал SQL-запрос, выводящий 5 шаблонов, которые чаще всего применяются юзерами 2 и более раза подряд в течение одной сессии.

In [2]:
# Если база данных ещё не создана, то создадим
if os.path.exists('logs.db') is not True:
    con = sqlite3.connect("logs.db")
    cur = con.cursor()
    # Создание таблицы, соответствующей условию
    cur.execute("""CREATE TABLE logs(
        user_id INT,
        event TEXT,
        event_time DATE,
        value TEXT);
    """)
    con.commit()
    cur.close()
    con.close()

# Теперь заполним таблицу данными, соответствующими условию.
# Например, добавим в неё 9 пользователей, 2 события event,
# время в диапозоне часа и 8 значений value

n = 300  # Пусть n - колличество записей в базе данных
user_id = [1,2,3,4,5,6,7,8,9]  # для заполнения столбца user_id
event = ['template_selected','another_template']  # для заполнения столбца event
value = ['value1','value2','value3','value4','value5','value6','value7','value8']  # для заполнения столбца value

# для заполнения столбца event_time
event_time = [] 

def random_time(start, end):
    # Эта функция вернёт случайное время из промежутка
    delta = end - start
    random_second = randrange((delta.days * 24 * 60 * 60) + delta.seconds)
    return start + timedelta(seconds=random_second)

d1 = datetime.strptime('2021-3-25 15:30:00', '%Y-%m-%d %H:%M:%S')
d2 = datetime.strptime('2021-3-25 16:30:00', '%Y-%m-%d %H:%M:%S')

for i in range(n):
    event_time.append(str(random_time(d1, d2)))

table = []
for i in range(n):
    table.append((
        np.random.randint(0,10),
        event[np.random.randint(len(event))],
        event_time[i],
        value[np.random.randint(len(value))] ))  

# Теперь заполним таблицу подобранными значениями
con = sqlite3.connect("logs.db")
cur = con.cursor()
cur.execute("DELETE FROM logs")  # Чтобы при повторном запуске кол-во строк не менялось
cur.executemany("INSERT INTO logs VALUES (?,?,?,?)", table)
con.commit()
df = pd.read_sql_query("SELECT * FROM logs", con)
cur.close()
con.close()
df.tail(10)


,user_id,event,event_time,value
290,4,template_selected,2021-03-25 15:47:29,value6
291,7,template_selected,2021-03-25 16:07:46,value2
292,8,template_selected,2021-03-25 16:18:49,value1
293,0,another_template,2021-03-25 16:01:55,value3
294,1,another_template,2021-03-25 15:47:37,value8
295,1,another_template,2021-03-25 16:22:43,value5
296,8,template_selected,2021-03-25 15:35:28,value1
297,0,template_selected,2021-03-25 16:16:52,value5
298,7,another_template,2021-03-25 16:00:49,value7
299,8,template_selected,2021-03-25 15:55:08,value4


In [3]:
# SQL-запрос, выводящий 5 шаблонов, которые чаще всего применяются
# юзерами 2 и более раза подряд в течение одной сессии
con = sqlite3.connect('logs.db')
cur = con.cursor()

answer = pd.read_sql_query("""
SELECT templates as top_5
FROM
    (SELECT val_prev templates, count(*) frequency
    FROM
        (SELECT A.user_id, A.value val_prev, B.value val_next, A.event_time time_prev, B.event_time time_next, 
        strftime('%s', B.event_time) - strftime('%s', A.event_time) delta
        FROM 
            (SELECT user_id, value, event_time,
            row_number() OVER(PARTITION BY user_id ORDER BY user_id, event_time) as t
            FROM logs WHERE event = 'template_selected') as A
            
            LEFT JOIN (SELECT user_id, value, event_time, 
            row_number() OVER(PARTITION BY user_id ORDER BY user_id, event_time) as t
            FROM logs WHERE event = 'template_selected') as B
            
            ON A.user_id=B.user_id AND A.t=B.t-1)
    WHERE val_prev=val_next AND delta<=300
    GROUP BY templates
    ORDER BY frequency DESC)
LIMIT 5
""", con)

cur.close()
con.close()

In [4]:
# Теперь посмотрим на результат того, что получилось
answer.index += 1
answer

,top_5
1,value7
2,value6
3,value4
4,value1
5,value8
